<a href="https://colab.research.google.com/github/JC1201/Intelligent-Edge-Caching-Using-Federated-Learning-to-Predict-Content-Popularity/blob/main/TYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import tensorflow_datasets as tfds
import numpy
from tensorflow.python.ops.gen_dataset_ops import shuffle_dataset
from sklearn.model_selection import train_test_split
import random

dataset, info = tfds.load("movielens/100k-ratings", split=["train"], with_info=True)


tf.random.set_seed(42)
numpy.random.seed(42)
random.seed(42)

data_set = dataset[0]

def preprocess_data(dataset):

    user_id = tf.strings.to_number(dataset["user_id"], out_type=tf.int32)
    movie_id = tf.strings.to_number(dataset["movie_id"], out_type=tf.int32)

    #casting user rating into 0 and 1 (0 means didn't rate this movie, 1 means did rate this movie)

    did_rate = tf.cast(dataset["user_rating"] > 0, tf.int64)

    features = {
        "user_id": user_id,
        "movie_id": movie_id,
        "user_rating":  did_rate
    }

    return features

data_set = data_set.map(preprocess_data)

print(len(data_set))

num_client = 5
userS_dataset = []

dataset_size = len(data_set)

for i in range(num_client):

    seed = numpy.random.seed(i)
    data_set = data_set.shuffle(buffer_size=dataset_size, seed= seed)
    userS_dataset.append(data_set)

data_set_1 =  data_set.shuffle(buffer_size=dataset_size, seed=42)

100000


In [2]:
#size of training data and testing data
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

#extract training data and testing data
train_data_1 = data_set_1.take(int(train_size))
test_data_1 = data_set_1.skip(int(train_size))

train_data = []
test_data = []

for i in range(num_client):

    train_data.append(userS_dataset[i].take(int(train_size)))
    test_data.append(userS_dataset[i].skip(int(train_size)))


In [3]:
userIDs = []
movieIDs = []
ratings = []

unique_user_movie_pair = set()
number = 0

for example in data_set:

    # Convert TensorFlow tensor to a NumPy value
    user_id = example["user_id"].numpy()
    movie_id = example["movie_id"].numpy()
    rating = example["user_rating"].numpy()

    # Add movie pair only if it is uniuqe
    if (user_id, movie_id) not in unique_user_movie_pair:
        userIDs.append(user_id)
        movieIDs.append(movie_id)
        ratings.append(rating)
        unique_user_movie_pair.add((user_id, movie_id))
        number += 1

print(f"Number of unique movie pairs: {number}")

# Convert lists to NumPy arrays for later processing
userIDs = numpy.array(userIDs)
movieIDs = numpy.array(movieIDs)
ratings = numpy.array(ratings)

movieIDs = numpy.unique(movieIDs)
userIDs = numpy.unique(userIDs)

#size of user and movie in training data
num_user = len(userIDs)
num_movie = len(movieIDs)
rating = len(ratings)

print(f"Number of unique users: {num_user}")
print(f"Number of unique movies: {num_movie}")
print(f"Number of ratings: {rating}")


Number of unique movie pairs: 100000
Number of unique users: 943
Number of unique movies: 1682
Number of ratings: 100000


In [4]:
triplets = list(zip(userIDs, movieIDs, ratings))

train_triplets, test_triplets = train_test_split(triplets, test_size=0.2, random_state=50)
train_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)
test_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)

def integrate_feature_into_matrix(userIDs, movieIDs, ratings, num_user, num_movie, matrix):

    # Create a 2D matrix filled with zeros
    # Populate the matrix
    for userID, movieID, rating in zip(userIDs, movieIDs, ratings):
        matrix[int(userID), int(movieID)] = rating

    return matrix

for user_id, movie_id, rating in train_triplets:
    train_matrix[user_id-1, movie_id-1] = rating

for user_id, movie_id, rating in test_triplets:
    test_matrix[user_id-1, movie_id-1] = rating

print(train_matrix.shape)
print(test_matrix.shape)


print(train_matrix.shape)
print(test_matrix.shape)

movie_popularity = numpy.sum(train_matrix, axis=0)
movie_popularity_normalized = movie_popularity / numpy.max(movie_popularity)
movie_label = movie_popularity_normalized.reshape((1, movie_popularity_normalized.shape[0]))

train_matrix = train_matrix.reshape((1, num_user, num_movie, 1))


(943, 1682)
(943, 1682)
(943, 1682)
(943, 1682)


In [5]:
# X_train_user, X_test_user, X_train_movie, X_test_movie, y_train, y_test = train_test_split(
#     userIDs, movieIDs, ratings, test_size=0.2, random_state=42
# )

In [6]:
def build_cnn_model(num_user, num_movie):

    regularization = l2(0.002)

    input_matrix = tf.keras.layers.Input(shape=(num_user, num_movie, 1), name="user_id")

    # First Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(128, kernel_size=(3, 3),strides = (1,1), padding = "SAME", kernel_regularizer=regularization, use_bias = False)(input_matrix)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.Activation('relu')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (1,1),)(cnn_layer)
    # cnn_layer = tf.keras.layers.Dropout(0.3)(cnn_layer)

    # Second Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides = (1,1), padding = "SAME", kernel_regularizer=regularization, use_bias = False)(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.SeparableConv2D(64, (3, 3), padding='same',  use_bias=False)(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.Activation('relu')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (2,2),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.2, seed = 42)(cnn_layer)

    # Third Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(2, 2), strides = (1,1), padding = "SAME", kernel_regularizer=regularization, use_bias = False)(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.SeparableConv2D(64, (3, 3), padding='same', use_bias=False)(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.Activation('relu')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides = (2,2),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.3, seed = 42)(cnn_layer)

    # cnn_layer = tf.keras.layers.GlobalAveragePooling2D()(cnn_layer)
    # cnn_layer = tf.keras.layers.GlobalAveragePooling2D()(cnn_layer)

    #cnn_layer = tf.keras.layers.Dense(128, activation='relu')(cnn_layer)
    #cnn_layer = tf.keras.layers.Dense(64, activation='relu')(cnn_layer)

    # cnn_layer = tf.keras.layers.Reshape((1, 1, 64))(cnn_layer)

    cnn_layer = tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=1, padding="SAME", activation="relu")(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (1,1),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.3, seed = 42)(cnn_layer)

    cnn_layer = tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=1, padding="SAME", activation="relu")(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (2,2),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.3,  seed = 42)(cnn_layer)

    cnn_layer = tf.keras.layers.Conv2DTranspose(64, kernel_size=2, strides=1, padding="SAME", activation="relu")(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides = (2,2),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.4, seed = 42)(cnn_layer)

    cnn_layer = tf.keras.layers.Flatten()(cnn_layer)

    # Dense Layer
    dense_layer = tf.keras.layers.Dense(64, activation='relu')(cnn_layer)

    # dense_layer = tf.keras.layers.Dropout(0.3)(dense_layer)

    dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

    # dense_layer = tf.keras.layers.Dropout(0.6)(dense_layer)

    dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

    #dense_layer = tf.keras.layers.Dropout(0.5)(dense_layer)

    # Output Layer
    output = tf.keras.layers.Dense(num_movie, activation='sigmoid', name="movie_scores")(cnn_layer)

    # Build and Compile the Model
    model = tf.keras.models.Model(inputs=input_matrix, outputs=output, name="MoviePopularityModel")

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=5000,  # Slower decay
        decay_rate=0.98,
        staircase=True
    )

    def aggregated_popularity_loss(y_true, y_pred):

        # Remove the last dimension: (batch_size, num_users, num_movies)
        y_true_squeezed = tf.squeeze(y_true, axis=-1)

        # Aggregate over the user axis (axis=1). Here we use sum, but you could also use tf.reduce_mean.
        true_popularity = tf.reduce_sum(y_true_squeezed, axis=1)  # Shape: (batch_size, num_movies)
        # changed from reduce sum to reduce mean

        # Normalize the aggregated popularity.
        # For each sample in the batch, divide by the maximum popularity value to bring scores to [0, 1].
        true_popularity_norm = (true_popularity - tf.reduce_min(true_popularity)) / (tf.reduce_max(true_popularity) - tf.reduce_min(true_popularity) + 1e-7)

        # Calculate the mean squared error between normalized true popularity and predictions.
        loss = tf.reduce_mean(tf.square(true_popularity_norm - y_pred))

        return loss

    def ranking_loss(y_true, y_pred):
        """Pairwise Ranking Loss: Higher-ranked items should have higher scores"""
        y_true = tf.squeeze(y_true, axis=-1)
        true_popularity = tf.reduce_sum(y_true, axis=1)

        diff = tf.expand_dims(true_popularity, -1) - tf.expand_dims(true_popularity, -2)
        pred_diff = tf.expand_dims(y_pred, -1) - tf.expand_dims(y_pred, -2)

        loss = tf.reduce_mean(tf.nn.relu(diff * pred_diff))
        return loss

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate = 0.002, rho = 0.85), #, epsilon=1e-7),
        # optimizer=tf.keras.optimizers.AdamW(learning_rate = 0.0005),
        #loss = ranking_loss,
        loss= aggregated_popularity_loss,
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

    return model

# Build the model
model = build_cnn_model(num_user, num_movie)

# Train the model
model.fit(
    train_matrix,
    train_matrix,
    #movie_label,
    epochs= 100,
)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - loss: 0.7151 - root_mean_squared_error: 0.6420
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - loss: 0.3600 - root_mean_squared_error: 0.6555
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - loss: 0.3048 - root_mean_squared_error: 0.6675
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - loss: 0.2891 - root_mean_squared_error: 0.6681
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step - loss: 0.2792 - root_mean_squared_error: 0.6695
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - loss: 0.2716 - root_mean_squared_error: 0.6692
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step - loss: 0.2641 - root_mean_squared_error: 0.6692
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step - loss: 0.2559 - root_mean_squared_error: 0.6690
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - loss: 0.2482 - root_mean_squared_error: 0.6696
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - loss: 0.2406 - root_mean_squared_error: 0.6693


In [7]:
test_matrix_reshaped = test_matrix.reshape((1, num_user, num_movie, 1))

# Predict movie scores
predictions = model.predict(test_matrix_reshaped)

# Aggregate scores across users
movie_scores = predictions.mean(axis=0)

# Get top 50 movie indices
# Top 50 in descending ordere
top_movies = numpy.argsort(-movie_scores)[:]  # Negative sign to get descending order

print("Predicted Movies Ranking(by index):")
for rank, movie_idx in enumerate(top_movies, 1):
    print(f"{rank}: Movie Index {movie_idx}, Score {movie_scores[movie_idx] } ")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step
Predicted Movies Ranking(by index):
1: Movie Index 1242, Score 0.9987800717353821 
2: Movie Index 1319, Score 0.9987512826919556 
3: Movie Index 1508, Score 0.9986020922660828 
4: Movie Index 1178, Score 0.9973214268684387 
5: Movie Index 676, Score 0.9971334934234619 
6: Movie Index 1506, Score 0.9969178438186646 
7: Movie Index 944, Score 0.9966748952865601 
8: Movie Index 1377, Score 0.9966714382171631 
9: Movie Index 1179, Score 0.9956516623497009 
10: Movie Index 1406, Score 0.9946012496948242 
11: Movie Index 183, Score 0.9938158988952637 
12: Movie Index 1399, Score 0.9938054084777832 
13: Movie Index 1185, Score 0.9919706583023071 
14: Movie Index 194, Score 0.9910308122634888 
15: Movie Index 1277, Score 0.9899105429649353 
16: Movie Index 1048, Score 0.9886935353279114 
17: Movie Index 1033, Score 0.9878589510917664 
18: Movie Index 1584, Score 0.9866085648536682 
19: Movie Index 1411, Score 0.9800184369087219 
20: Movie Index 1608, Scor

In [8]:
def calculate_caching_hit_rate(predicted_scores, test_matrix, cache_size_array):

    """
    Calculate the caching hit rate.

    :param predicted_scores: Array of predicted scores for all movies (shape: num_movies).
    :param test_data: Test dataset containing actual movie requests (e.g., movieIDs).
    :param cache_size: Number of movies to cache (top-k based on predicted scores).
    :return: Hit rate as a float.
    """

    predicted_scores = predicted_scores.flatten()

    num_user, num_movie = test_matrix.shape

    for cache_size in cache_size_array:

      # Get top-k movies based on predicted scores
      top_k_movies = numpy.argsort(predicted_scores)[-cache_size:][::-1]

      #Extract actual requested movie IDs from test_matrix
      requested_movie_ids = set()
      hits = 0

      for user_id in range(num_user):

          requested_movies = numpy.where(test_matrix[user_id] == 1)[0]
          requested_movie_ids.update(requested_movies)

      # Calculate hit rate
      for movie_id in requested_movie_ids:
          if movie_id in top_k_movies:
              hits += 1

      total_unique_movies = len(requested_movie_ids)
      print(f"Total unique movies: {total_unique_movies}, total hit: {hits}")

      hit_rate = hits / cache_size
      print(f"Hit Rate for cache size {cache_size}: {hit_rate:.2%}")
      print("")

    return hit_rate

# Example usage
cache_size_array = [50, 100, 150, 200, 250, 300]  # the number of cached top N movies
hit_rate = calculate_caching_hit_rate(predictions, test_matrix, cache_size_array)

Total unique movies: 189, total hit: 10
Hit Rate for cache size 50: 20.00%

Total unique movies: 189, total hit: 20
Hit Rate for cache size 100: 20.00%

Total unique movies: 189, total hit: 29
Hit Rate for cache size 150: 19.33%

Total unique movies: 189, total hit: 40
Hit Rate for cache size 200: 20.00%

Total unique movies: 189, total hit: 51
Hit Rate for cache size 250: 20.40%

Total unique movies: 189, total hit: 62
Hit Rate for cache size 300: 20.67%

